# Sampling

## Get Data

In [ ]:
import torch
from data.getDataset import getCognatesSet, getIteration
from data.vocab import computeInferenceData, V_SIZE
from torch.nn.utils.rnn import pad_sequence
from Types.articleModels import ModernLanguages
from Types.models import InferenceData
from Source.reconstructionModel import ReconstructionModel

device = 'cuda' if torch.cuda.is_available() else 'cpu'

cognates:dict[ModernLanguages, InferenceData] = {lang:computeInferenceData(pad_sequence(cogSet, batch_first=True)) for (lang, cogSet) in getCognatesSet().items()}
currentReconstructions = computeInferenceData(pad_sequence(getIteration(1), batch_first=True))

voc_size = V_SIZE
LSTM_INPUT_DIM = 50
LSTM_HIDDEN_DIM = 50

randomEditModel = ReconstructionModel(cognates, voc_size, LSTM_INPUT_DIM, LSTM_HIDDEN_DIM)

x_maxLength = currentReconstructions[0].size()[0] - 2
print('|x| max =', x_maxLength)

False Language Model with neutral probability for test

In [ ]:
from torch import Tensor
from Types.models import InferenceData
from lm.PriorLM import PriorLM
from numpy import ndarray

class LM(PriorLM):
    def __init__(self):
        pass
    def inference(self, reconstructions: InferenceData) -> ndarray:
        return torch.ones((len(reconstructions[1]),)).numpy()
    
lm = LM()

## Generation

In [ ]:
from Source.generateProposals import generateProposalsFromCurrentReconstructions
from torch.nn.utils.rnn import unpad_sequence

proposalsList = generateProposalsFromCurrentReconstructions(
    unpad_sequence(currentReconstructions[0][1:], currentReconstructions[1]),
    getCognatesSet()
)
torch.save(proposalsList, './Tests/proposalsSet.pt')

In [ ]:
proposalsList = torch.load('./Tests/proposalsSet.pt')

## Sampling from proposals

In [ ]:
from Source.sampling import metropolisHasting

samples = metropolisHasting(proposalsList, randomEditModel, lm)